In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer

Import dataset yang telah dilakukan tahap preprocessing

In [8]:
df_students = pd.read_csv("/Users/althaaframadhan/Documents/Skripsi/penelitian/preprocessing/students_200.csv")
df_trends   = pd.read_csv("/Users/althaaframadhan/Documents/Skripsi/penelitian/preprocessing/trends_200.csv")


menampilkan 5 baris dari data students asli

In [9]:
print(df_students.head())

   Age  Gender Academic_Level     Country  Avg_Daily_Usage_Hours  \
0   20  Female  Undergraduate      France                    3.0   
1   19  Female  Undergraduate  Azerbaijan                    4.8   
2   22  Female       Graduate  Kyrgyzstan                    2.9   
3   19  Female  Undergraduate  Bangladesh                    4.2   
4   22    Male       Graduate     Germany                    3.1   

  Most_Used_Platform Affects_Academic_Performance  Sleep_Hours_Per_Night  \
0          Instagram                           No                    9.0   
1             TikTok                          Yes                    5.7   
2           Facebook                           No                    7.0   
3          Instagram                          Yes                    7.8   
4           Facebook                           No                    8.4   

   Mental_Health_Score  Conflicts_Over_Social_Media  Addicted_Score  
0                    7                            2             

menampilkan 5 dataset baris dari dataset trends

In [10]:
print(df_trends.head())

    Platform     Hashtag Content_Type   Region    Views   Likes  Shares  \
0  Instagram     #Comedy        Video   Canada  3239663  288436   94297   
1  Instagram  #Education         Post    Japan  3759036   49515   94487   
2  Instagram     #Gaming       Shorts   Brazil  3697216  209784   41633   
3     TikTok    #Fitness         Post   Brazil  2878436  149765    9840   
4     TikTok    #Fashion  Live Stream  Germany  1213710  468518    1746   

   Comments Engagement_Level  
0      5399             High  
1     19525           Medium  
2     19831           Medium  
3     42118             High  
4     24547             High  


merubah nilai numerik menjadi kategorik dengan dicretization pada dataset students dengan nilai low, medium dan hiogh utnuk merepresentasikan nilai data nhumerik

In [11]:
df_b1= df_students.copy()
num_cols = df_b1.select_dtypes(include=['int64', 'float64']).columns
labels = ['Low', 'Medium', 'High']

for col in num_cols:
    disc = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
    try:
        disc.fit(df_b1[[col]].dropna())
    except Exception as e:
        print(f"⚠️ Gagal fit kolom {col}: {e}. Dilewati.")
        continue

    ord_vals = disc.transform(df_b1[[col]]).ravel()
    ord_int = pd.Series(
        np.where(np.isnan(ord_vals), np.nan, np.round(ord_vals).astype(int)),
        index=df_b1.index
    )

    mapping = {i: labels[i] for i in range(len(labels))}
    mapped = ord_int.map(mapping)

    # 🔹 Buat categorical Series baru terlebih dahulu
    cat_series = pd.Categorical(mapped, categories=labels, ordered=True)
    
    # 🔹 Ganti kolom secara aman via DataFrame.assign()
    df_b1 = df_b1.assign(**{col: cat_series})


print(df_b1.head())
print("\nTipe data kolom hasil:")
print(df_b1.dtypes)
df_b1.to_csv("/Users/althaaframadhan/Documents/Skripsi/penelitian/transformation/students_ntc.csv", index=False)



      Age  Gender Academic_Level     Country Avg_Daily_Usage_Hours  \
0  Medium  Female  Undergraduate      France                   Low   
1     Low  Female  Undergraduate  Azerbaijan                Medium   
2    High  Female       Graduate  Kyrgyzstan                   Low   
3     Low  Female  Undergraduate  Bangladesh                   Low   
4    High    Male       Graduate     Germany                   Low   

  Most_Used_Platform Affects_Academic_Performance Sleep_Hours_Per_Night  \
0          Instagram                           No                  High   
1             TikTok                          Yes                   Low   
2           Facebook                           No                Medium   
3          Instagram                          Yes                  High   
4           Facebook                           No                  High   

  Mental_Health_Score Conflicts_Over_Social_Media Addicted_Score  
0                High                      Medium            

melihat cakupan nilai lopw, medium, dan high pada nillai numerik data

In [12]:
df_b = df_students.copy()
num_cols = df_b.select_dtypes(include=['int64', 'float64']).columns
labels = ['Low', 'Medium', 'High']

bin_edges_info = {}

for col in num_cols:
    disc = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
    try:
        disc.fit(df_b[[col]].dropna())
    except Exception as e:
        print(f"⚠️ Gagal fit kolom {col}: {e}. Dilewati.")
        continue

    edges = disc.bin_edges_[0]
    bin_edges_info[col] = edges

def nice(x):
    if np.isclose(x, np.round(x)):
        return f"{int(round(x))}"
    else:
        return f"{x:.1f}"

print("=== Interpretasi Batas Nilai (per Kolom) ===")
for col, edges in bin_edges_info.items():
    print(f"\n📊 {col}:")
    for i in range(len(edges) - 1):
        print(f"  {labels[i]}  →  {nice(edges[i])}  ≤ nilai <  {nice(edges[i+1])}")


=== Interpretasi Batas Nilai (per Kolom) ===

📊 Age:
  Low  →  18  ≤ nilai <  20
  Medium  →  20  ≤ nilai <  21
  High  →  21  ≤ nilai <  24

📊 Avg_Daily_Usage_Hours:
  Low  →  1.5  ≤ nilai <  4.4
  Medium  →  4.4  ≤ nilai <  5.5
  High  →  5.5  ≤ nilai <  8.4

📊 Sleep_Hours_Per_Night:
  Low  →  3.9  ≤ nilai <  6.2
  Medium  →  6.2  ≤ nilai <  7.4
  High  →  7.4  ≤ nilai <  9

📊 Mental_Health_Score:
  Low  →  4  ≤ nilai <  6
  Medium  →  6  ≤ nilai <  7
  High  →  7  ≤ nilai <  9

📊 Conflicts_Over_Social_Media:
  Low  →  0  ≤ nilai <  2
  Medium  →  2  ≤ nilai <  3
  High  →  3  ≤ nilai <  5

📊 Addicted_Score:
  Low  →  2  ≤ nilai <  5.3
  Medium  →  5.3  ≤ nilai <  7
  High  →  7  ≤ nilai <  9


melkukan discretizatioin pada dataset trends

In [13]:
df_b2= df_trends.copy()
num_cols = df_b2.select_dtypes(include=['int64', 'float64']).columns
labels = ['Low', 'Medium', 'High']

for col in num_cols:
    disc = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
    try:
        disc.fit(df_b2[[col]].dropna())
    except Exception as e:
        print(f"⚠️ Gagal fit kolom {col}: {e}. Dilewati.")
        continue

    ord_vals = disc.transform(df_b2[[col]]).ravel()
    ord_int = pd.Series(
        np.where(np.isnan(ord_vals), np.nan, np.round(ord_vals).astype(int)),
        index=df_b2.index
    )

    mapping = {i: labels[i] for i in range(len(labels))}
    mapped = ord_int.map(mapping)

    # 🔹 Buat categorical Series baru terlebih dahulu
    cat_series = pd.Categorical(mapped, categories=labels, ordered=True)
    
    # 🔹 Ganti kolom secara aman via DataFrame.assign()
    df_b2 = df_b2.assign(**{col: cat_series})

print(df_b2.head())
print("\nTipe data kolom hasil:")
print(df_b2.dtypes)
df_b2.to_csv("/Users/althaaframadhan/Documents/Skripsi/penelitian/transformation/trends_ntc.csv", index=False)


    Platform     Hashtag Content_Type   Region   Views   Likes  Shares  \
0  Instagram     #Comedy        Video   Canada  Medium  Medium    High   
1  Instagram  #Education         Post    Japan    High     Low    High   
2  Instagram     #Gaming       Shorts   Brazil    High  Medium  Medium   
3     TikTok    #Fitness         Post   Brazil  Medium     Low     Low   
4     TikTok    #Fashion  Live Stream  Germany     Low    High     Low   

  Comments Engagement_Level  
0      Low             High  
1   Medium           Medium  
2   Medium           Medium  
3     High             High  
4   Medium             High  

Tipe data kolom hasil:
Platform              object
Hashtag               object
Content_Type          object
Region                object
Views               category
Likes               category
Shares              category
Comments            category
Engagement_Level      object
dtype: object


Menampilkan nilai discret low,medium, dan hiogh dalam representasi data numerik 

In [14]:
df_b = df_trends.copy()
num_cols = df_b.select_dtypes(include=['int64', 'float64']).columns
labels = ['Low', 'Medium', 'High']

bin_edges_info = {}

for col in num_cols:
    disc = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
    try:
        disc.fit(df_b[[col]].dropna())
    except Exception as e:
        print(f"⚠️ Gagal fit kolom {col}: {e}. Dilewati.")
        continue

    edges = disc.bin_edges_[0]
    bin_edges_info[col] = edges

def nice(x):
    if np.isclose(x, np.round(x)):
        return f"{int(round(x))}"
    else:
        return f"{x:.1f}"

print("=== Interpretasi Batas Nilai (per Kolom) ===")
for col, edges in bin_edges_info.items():
    print(f"\n📊 {col}:")
    for i in range(len(edges) - 1):
        print(f"  {labels[i]}  →  {nice(edges[i])}  ≤ nilai <  {nice(edges[i+1])}")


=== Interpretasi Batas Nilai (per Kolom) ===

📊 Views:
  Low  →  17410  ≤ nilai <  1783311
  Medium  →  1783311  ≤ nilai <  3506266
  High  →  3506266  ≤ nilai <  4949423

📊 Likes:
  Low  →  1349  ≤ nilai <  172432
  Medium  →  172432  ≤ nilai <  338794
  High  →  338794  ≤ nilai <  498119

📊 Shares:
  Low  →  869  ≤ nilai <  33122.3
  Medium  →  33122.3  ≤ nilai <  67844
  High  →  67844  ≤ nilai <  99902

📊 Comments:
  Low  →  275  ≤ nilai <  12019
  Medium  →  12019  ≤ nilai <  30492.3
  High  →  30492.3  ≤ nilai <  49438
